In [15]:
# Run the pip install command below if you don't already have the library
# !pip install git+https://github.com/JustAnotherArchivist/snscrape.git

import os
import math
import pandas as pd
import json
from pandas import json_normalize

import snscrape.modules.twitter as sntwitter
import itertools

from datetime import datetime
from random import randrange
import shutil

In [1]:
# Search terms: Content in the tweet
search_terms = [
    'coronavirus',
    'covid19',
    'pandemic',
    'quarantine',
]

# Creates a search parameter for the Twitter scraper
# since_date: inclusive
# until_date: exclusive
def search_builder(terms, city, since_date, until_date):    
    s = ''
    # Concatenate all search terms
    for i in range(len(terms) - 1):
        # NOTE: Inclusive OR statement
        s += '\"' + terms[i] + '\" OR '
    s += '\"' + terms[len(terms) - 1] + '\" '    
    
    # Within or the near the city
    s += 'near:\"' + city + '\" '
    
    #s += 'place:96683cc9126741d1 '

    
    # Since the first date
    s += 'since:' + since_date + ' '
    # Until the second date
    s += 'until:' + until_date
    
    return s

# Returns a DataFrame with at most tweet_limit tweets resulting from
# the search
def scrape(search_param, tweet_limit):
    return pd.DataFrame(itertools.islice(
        sntwitter.TwitterSearchScraper(search_param).get_items(), tweet_limit))

# Search for tweets in Los Angeles mentioning COVID terms
# from the beginning to the end of 2020.
los_angeles_search = search_builder(
    search_terms,
    "Los Angeles",
    '2020-01-01',
    '2020-12-31')

print(los_angeles_search)

"coronavirus" OR "covid19" OR "pandemic" OR "quarantine" near:"Los Angeles" since:2020-01-01 until:2020-12-31


In [74]:
tweet_limit = 10000
since_date = '2021-01-01'
until_date = '2021-02-01'

total_tweets = 0

# NOTE: Below process with 10000 items for California takes about 45 - 60 minutes to run

print("Compiling tweets (max <=" + str(tweet_limit) + ") from California counties from " + since_date + " to " + until_date)

print("START: " + str(datetime.now()))

ca_counties = pd.read_csv("ca_counties.csv")

for i in range(len(ca_counties)):
    
    county = ca_counties.iloc[i]['County']
    
    county_search = search_builder(
        search_terms,
        county + ", CA",
        since_date,
        until_date)
    
    county_tweets = scrape(county_search, tweet_limit)
    
    total_tweets = total_tweets + len(county_tweets)
    
    # Generates JSON with respective county tweet data
    county_json = county_tweets.to_json(county + ".json", lines=True, orient='records')
    
    # Moves file to data/ca_tweets/ directory
    source = os.getcwd() + "/" + county + ".json"
    destination = os.getcwd() + "/data/ca_tweets/" + county + ".json"
    shutil.move(source, destination)
    
    print("(" + str(i + 1) + "/" + str(len(ca_counties)) + ") " + county + " - " + str(len(county_tweets)) + " tweets: " + str(datetime.now()))
    
print("END: " + str(datetime.now()))
print(str(total_tweets) + " tweets processed")

Compiling tweets (max <=10000) from California counties from 2021-01-01 to 2021-02-01
START: 2021-10-20 17:19:55.811771
(1/58) Alameda County - 281 tweets: 2021-10-20 17:20:08.600761
(2/58) Alpine County - 10000 tweets: 2021-10-20 17:23:48.389175
(3/58) Amador County - 0 tweets: 2021-10-20 17:23:53.089564
(4/58) Butte County - 11 tweets: 2021-10-20 17:23:54.506167
(5/58) Calaveras County - 0 tweets: 2021-10-20 17:23:56.239812
(6/58) Colusa County - 10000 tweets: 2021-10-20 17:27:33.601771
(7/58) Contra Costa County - 182 tweets: 2021-10-20 17:27:42.303995
(8/58) Del Norte County - 0 tweets: 2021-10-20 17:27:43.763875
(9/58) El Dorado County - 9 tweets: 2021-10-20 17:27:45.482482
(10/58) Fresno County - 107 tweets: 2021-10-20 17:27:50.754506
(11/58) Glenn County - 10000 tweets: 2021-10-20 17:31:26.899605
(12/58) Humboldt County - 4 tweets: 2021-10-20 17:31:28.612773
(13/58) Imperial County - 20 tweets: 2021-10-20 17:31:30.448126
(14/58) Inyo County - 10000 tweets: 2021-10-20 17:35:06.18

# References
## California Counties and Cities: https://www.counties.org/cities-within-each-county